### Join CA Data
The Climate/Anthropogenic data accessed from Google Earth Engine in the `S1-Load_CA_Data` notebook is joined to the streamflow data in this notebook.

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import os
import datetime

In [ ]:
static_df = gpd.read_file('Data/basins/attributes.gpkg')

# join monthly water recurrence data
df = pd.read_csv('Data/gee/JRC-GSW1_4-MonthlyRecurrence__monthly_recurrence.csv')
df['month'] = df['system:index'].apply(lambda x: int(x.split('_')[2]))

static_df['jan_recurrence'] = np.nan
static_df['feb_recurrence'] = np.nan
static_df['mar_recurrence'] = np.nan
static_df['apr_recurrence'] = np.nan
static_df['may_recurrence'] = np.nan
static_df['jun_recurrence'] = np.nan
static_df['jul_recurrence'] = np.nan
static_df['aug_recurrence'] = np.nan
static_df['sep_recurrence'] = np.nan
static_df['oct_recurrence'] = np.nan
static_df['nov_recurrence'] = np.nan
static_df['dec_recurrence'] = np.nan

month_dict = {0:'jan_recurrence', 1:'feb_recurrence', 2:'mar_recurrence',
              3:'apr_recurrence', 4:'may_recurrence', 5:'jun_recurrence',
              6:'jul_recurrence', 7:'aug_recurrence', 8:'sep_recurrence',
              9:'oct_recurrence', 10:'nov_recurrence', 11:'dec_recurrence'}

for _, r in df.iterrows():
    df_col = month_dict[r.month]
    static_df.loc[vote_df['index'] == r['index'], df_col] = r['mean']
    
static_df.to_csv('Data/basins/attributes/all.csv', index=False)

vote_df = pd.read_csv('Data/basins/attributes/all.csv')
files = os.listdir('Data/basins/time_series')

for i in files:
    i = str(i.split('.')[0])
    nc_xr = xr.open_dataset('Data/basins/time_series/' + i + '.nc')
    nc_df = nc_xr.to_dataframe()
    nc_df['month'] = nc_df.index.month
    nc_df['recurrence'] = nc_df.apply(lambda x: vote_df.loc[vote_df['index'] == int(i), month_dict[int(x.month) - 1]].item(), axis=1)
    nc_df.to_xarray().to_netcdf('Data/basins/time_series/' + i + '.nc')

In [ ]:
# join LCMAP time series
df = pd.read_csv('Data/gee/projects-sat-io-open-datasets-LCMAP-LCPRI__b1.csv')
df['year'] = df['system:index'].apply(lambda x: int(x.split('_')[2]))

hist_dict = {1: 'developed', 2: 'cropland', 3: 'grass_shrub', 4: 'tree_cover',
             5: 'water', 6: 'wetland', 7: 'ice_snow', 8: 'barren'}

df['developed'] = np.nan
df['cropland'] = np.nan
df['grass_shrub'] = np.nan
df['tree_cover'] = np.nan
df['water'] = np.nan
df['wetland'] = np.nan
df['ice_snow'] = np.nan
df['barren'] = np.nan

for _, r in df.iterrows():
    hist_list = r.histogram.split('{')[1].split('}')[0].split(',')
    
    for i in hist_list:
        df_var = hist_dict[int(i.split('=')[0])]
        df.loc[df['system:index'] == r['system:index'], df_var] = float(i.split('=')[1])
        
df = df.fillna(0.0)

files = os.listdir('Data/basins/time_series')

for f in files:
    nc_xr = xr.open_dataset('Data/basins/time_series/'+f)
    f_gage = f.split('.')[0]
    df_temp = df.loc[df['index'] == int(f_gage)][['index', 'year', 'developed', 'cropland', 'grass_shrub', 'tree_cover',
                                                     'water', 'wetland', 'ice_snow', 'barren']]
    df_temp.index = pd.to_datetime(pd.DataFrame({'year': df_temp.year, 'month': 1, 'day': 1}))
    df_temp = df_temp.resample('D').mean().fillna(method='ffill')
    df_temp.index.rename('date',inplace=True)
    xr_gage = df_temp.to_xarray()
    nc_xr = nc_xr.merge(xr_gage, join='left')
    nc_xr.to_netcdf('Data/basins/time_series/'+f)

In [ ]:
# join rangeland agriculture data
df = pd.read_csv('Data/gee/projects-rap-data-365417-assets-vegetation-cover-v3.csv')
df['year'] = df['system:index'].apply(lambda x: int(x.split('_')[0]))

hist_dict = {'AFG_mean': 'annual_forbs_grass', 'PFG_mean': 'perenn_forbs_grass', 'SHR_mean': 'shrubs',
             'TRE_mean': 'trees', 'BGR_mean': 'bare_ground'}

for i in hist_dict:
    df[hist_dict[i]] = df[i]
    
files = os.listdir('Data/basins/time_series')

for f in files:
    nc_xr = xr.open_dataset('Data/basins/time_series/'+f)
    f_gage = f.split('.')[0]
    df_temp = df.loc[df['index'] == int(f_gage)][['index', 'year', 'annual_forbs_grass', 'perenn_forbs_grass', 'shrubs', 'trees', 'bare_ground']]
    df_temp.index = pd.to_datetime(pd.DataFrame({'year': df_temp.year, 'month': 1, 'day': 1}))
    df_temp = df_temp.resample('D').mean().fillna(method='ffill')
    df_temp.index.rename('date',inplace=True)
    xr_gage = df_temp.to_xarray()
    nc_xr = nc_xr.merge(xr_gage, join='left')
    nc_xr.to_netcdf('Data/basins/time_series/'+f)

In [ ]:
# join random variable data
files = os.listdir('Data/basins/time_series')

for f in files:
    nc_df = xr.open_dataset('Data/basins/time_series/'+f).to_dataframe()
    nc_df['rand_n'] = np.random.rand(len(nc_df))
    nc_df.to_xarray().to_netcdf('Data/basins/time_series/'+f)

In [ ]:
# join hydro atlas data
for i in static_df['index']:
    nc_df = xr.open_dataset('Data/basins/time_series/'+str(i)+'.nc').to_dataframe()
    aet, pet, pre, cmi, pnv = [], [], [], [], []
    for n in range(1, 12+1):
        name = 'ha_aet_mm_s' + str(n).zfill(2)
        aet.append(static_df.loc[static_df['index'] == int(i), name].item())
        
        name = 'ha_pet_mm_s' + str(n).zfill(2)
        pet.append(static_df.loc[static_df['index'] == int(i), name].item())
        
        name = 'ha_pre_mm_s' + str(n).zfill(2)
        pre.append(static_df.loc[static_df['index'] == int(i), name].item())
        
        name = 'ha_cmi_ix_s' + str(n).zfill(2)
        cmi.append(static_df.loc[static_df['index'] == int(i), name].item())
        
        name = 'ha_pnv_pc_s' + str(n).zfill(2)
        pnv.append(static_df.loc[static_df['index'] == int(i), name].item())

    aet_df = pd.DataFrame({'month':list(np.arange(1.0, 13.0)),
                           'aet':aet, 'pet':pet, 'pre':pre, 'cmi':cmi, 'pnv':pnv})

    nc_df['aet'] = np.nan
    nc_df['pet'] = np.nan
    nc_df['pre'] = np.nan
    nc_df['cmi'] = np.nan
    nc_df['pnv'] = np.nan
    for idx, r in aet_df.iterrows():
        m = r.month
        nc_df.loc[nc_df.month == m, 'aet'] = r.aet
        nc_df.loc[nc_df.month == m, 'pet'] = r.pet
        nc_df.loc[nc_df.month == m, 'pre'] = r.pre
        nc_df.loc[nc_df.month == m, 'cmi'] = r.cmi
        nc_df.loc[nc_df.month == m, 'pnv'] = r.pnv

    nc_df.to_xarray().to_netcdf('Data/basins/time_series/'+str(i)+'.nc')

## Filter USGS Gauges

In [ ]:
# split into natural/undammed + post dam
basins = os.listdir('Data/basins/time_series/')

for i in static_df['index']:
    if str(i) + '.nc' in basins:
        nc_xr = xr.open_dataset("Data/basins/time_series/"+str(i)+".nc")
        if static_df.loc[static_df['index'] == i].nid_name.item() == '':
            # undammed
            nc_xr.to_netcdf('Data/basins/natural/'+str(i)+'.nc')
        else:
            yr = int(static_df.loc[static_df['index'] == i, 'yr_completed'].item())
            # pre dam
            if datetime.datetime(nc_xr.date.min().dt.year.to_numpy(), 1, 1) < datetime.datetime(int(yr), 1, 1):
                nc_xr_pre = nc_xr.sel(date=slice(nc_xr.date.min().to_numpy(), str(yr)+'-01-01'))
                nc_xr_pre.to_netcdf('Data/basins/natural/'+str(i)+'.nc')

            # post dam
            if datetime.datetime(nc_xr.date.max().dt.year.to_numpy(), 1, 1) > datetime.datetime(int(yr), 1, 1):
                nc_xr_post = nc_xr.sel(date=slice(str(yr)+'-01-01', nc_xr.date.max().to_numpy()))
                nc_xr_post.to_netcdf('Data/basins/dam/'+str(i)+'.nc')

natural = os.listdir('Data/basins/natural')
dam = os.listdir('Data/basins/dam')

natural_basins, dam_basins = [], []

for b in all_basins[0]:
    basin = str(b) + '.nc'
    
    if basin in natural:
        nc_df = xr.open_dataset('Data/basins/natural/' + basin).to_dataframe()
        if len(nc_df['2000-10-01':'2020-10-01'].dropna(subset='q_cms')) >= 365 * 12:
            natural_basins.append(b)

    if basin in dam:
        nc_df = xr.open_dataset('Data/basins/dam/' + basin).to_dataframe()
        if len(nc_df['2000-10-01':'2020-10-01'].dropna(subset='q_cms')) >= 365 * 12:
            dam_basins.append(b)

flashy_basins, natural_b, dam_b = [], [], []
for i in natural_basins:
    basin = str(i) + '.nc'
    nc_df = xr.open_dataset('Data/basins/natural/' + basin).to_dataframe()
    nc_df = nc_df['2000-10-01':'2020-10-01']
    if len(nc_df.loc[nc_df.q_cms < 0.01])/len(nc_df) >= 0.4:
        flashy_basins.append(i)
#         plt.plot(nc_df.q_cms)
#         plt.show()
    else:
        natural_b.append(i)

for i in dam_basins:
    basin = str(i) + '.nc'
    nc_df = xr.open_dataset('Data/basins/dam/' + basin).to_dataframe()
    nc_df = nc_df['2000-10-01':'2020-10-01']
    if len(nc_df.loc[nc_df.q_cms < 0.01])/len(nc_df) >= 0.4:
        flashy_basins.append(i)
#         plt.plot(nc_df.q_cms)
#         plt.show()
    else:
        dam_b.append(i)
        
static_df['split_group'] = np.nan

for i in all_basins[0]:
    if i in flashy_basins:
        att_df.loc[att_df['index'] == i, 'split_group'] = 'flashy'
    if i in natural_b:
        att_df.loc[att_df['index'] == i, 'split_group'] = 'natural'
    if i in dam_b:
        att_df.loc[att_df['index'] == i, 'split_group'] = 'dam'
        
static_df.to_csv('Data/basins/attributes/all.csv', index=False)